In [174]:
import requests
from requests.adapters import HTTPAdapter, Retry
import pandas as pd
import pprint as pp
from bs4 import BeautifulSoup as bs
import spacy
import json
import time

nlp = spacy.load('en_core_web_trf')


In [4]:
books3_df = pd.read_json('books3.json', orient='records', lines=True)
books3_df.head()

,country_code,language_code,average_rating,description,authors,publisher,num_pages,publication_year,image_url,ratings_count,book_id,work_id,title,genres,author_string,author_title,match
0,US,eng,3.49,Addie Downs and Valerie Adler were eight when ...,[Jennifer Weiner],Atria Books,368,2009,https://s.gr-assets.com/assets/nophoto/book/11...,51184,6066819,6243154,Best Friends Forever,"[fiction, romance, mystery, thriller, crime]",Jennifer Weiner,Jennifer Weiner - Best Friends Forever,True
1,US,eng,3.82,,[Tom Wolfe],,,,https://s.gr-assets.com/assets/nophoto/book/11...,77,89373,1080201,The Bonfire of the Vanities,"[fiction, mystery, thriller, crime, history, h...",Tom Wolfe,Tom Wolfe - The Bonfire of the Vanities,True
2,US,eng,4.26,What is Heaven really going to be like? What w...,[Randy Alcorn],,533,,https://images.gr-assets.com/books/1406508230m...,7345,89376,86257,Heaven,[non-fiction],Randy Alcorn,Randy Alcorn - Heaven,True
3,US,,3.98,It's 1953 and 11-year-old Penny dreams of a su...,[Jennifer L. Holm],Random House Books for Young Readers,288,2006,https://images.gr-assets.com/books/1320470906m...,6949,89377,86258,Penny from Heaven,"[fiction, history, historical fiction, biograp...",Jennifer L. Holm,Jennifer L. Holm - Penny from Heaven,True
4,US,,3.65,Comment quitte-t-on quelqu'un ? Et pourquoi ? ...,[Francoise Sagan],,,,https://images.gr-assets.com/books/1366284322m...,12,2008911,1040345,Silken Eyes,"[fiction, romance]",Francoise Sagan,Francoise Sagan - Silken Eyes,True


In [5]:
authors = books3_df['author_string'].to_list()

In [176]:
headers = {
    'User-Agent': 'Research on Datasets (esuhr@gradcenter.cuny.edu)',
}

def make_request(url):
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    response = session.get(url, headers=headers)
    parsed = response.json()
    return parsed

In [158]:
def findpages(author_name):
    author_name = author_name.replace(' ', '%20')
    url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch={author_name}"
    data = make_request(url)
    if len(data['query']['search']) > 0:
        title = data['query']['search'][0]['title']
        return title

In [142]:
def gethtml(title):
    url = f"http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&titles={title}&rvsection=0&rvparse"
    data = make_request(url)
    page_one = next(iter(data['query']['pages'].values()))
    if 'revisions' in page_one:
        html = next(iter(page_one['revisions'][0].values()))
        return html
    else:
        return None

In [172]:
def getdatelocs(html):
    if html is None:
        return None, None
    else:
        soup = bs(html, 'html.parser')
        # find span class="bday"
        infobox = soup.find('td', {'class': 'infobox-data'})
        if infobox:
            info = ''.join(infobox.text)
            info = info.replace('(', ' ').replace(')', ' ')

            infolabels = nlp(info)
            if 'DATE' in [ent.label_ for ent in infolabels.ents] and 'GPE' in [ent.label_ for ent in infolabels.ents]:
                date = [ent.text for ent in infolabels.ents if ent.label_ == 'DATE'][0]
                location = [ent.text for ent in infolabels.ents if ent.label_ == 'GPE']
                locationstr = ', '.join([str(loc) for loc in location])
                return date, locationstr
            else:
                return None, None
        else:
            return None, None

In [177]:
authorinfo = []
for i in range(len(authors)):
    authordict = {}
    author = authors[i]
    time.sleep(1)
    title = findpages(author)
    time.sleep(1)
    html = gethtml(title)
    date, location = getdatelocs(html)
    print(f'{i}:{author} was born on {date} in {location}')
    authordict['author'] = author
    authordict['bday'] = date
    authordict['location'] = location
    with open('authorinfo.json', 'a') as f:
        json.dump(authordict, f)
        f.write('\n')

0:Jennifer Weiner was born on 1970-03-28 in DeRidder, Louisiana, U.S.
1:Tom Wolfe was born on None in None
2:Randy Alcorn was born on None in None
3:Jennifer L. Holm was born on 1968-06-16 in San Diego, California, U.S.
4:Francoise Sagan was born on None in None
5:Peter Maass was born on None in None
6:Jane Austen was born on 1775-12-16 in Hampshire, England
7:Ernest Hemingway, Will Patton was born on 1954-06-14 in South Carolina, U.S.
8:Ian Botham was born on 1955-11-24 in Heswall, England
9:Mandy Stanley was born on 1952-11-30 in Chicago, Illinois, U.S.
10:Maya Cross was born on None in None
11:Beatrix Potter, Harry Deane Willmott was born on None in None
12:John  Langan was born on 1969-07-06 in United States
13:Catherine McKenzie was born on 1967-10-24 in Sydney, New South Wales, Australia
14:Michel Faber was born on 1960-04-13 in The Hague, Netherlands
15:Thomas F. Monteleone was born on 1946-04-14 in Baltimore, Maryland, U.S.
16:Anton Chekhov was born on 1860-01-29 in Ekaterinosl

ConnectionError: HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?action=query&prop=revisions&rvprop=content&format=json&titles=Emilia%20Fox&rvsection=0&rvparse (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x2efc2d5d0>: Failed to establish a new connection: [Errno 65] No route to host'))